## Lauren Turner (npa5gv) 

# DS 2002 Midterm Project - Medical Data Set Analysis

This synthetic medical dataset includes information on patients and their various appointments, conditions, immunizations, care plans, medications, and procedures that they have undergone. The goal of this Jupyter notebook is to manipulate the set of data in order to create an ETL pipeline between the MongoDB and MySQL databases.

### Importing Libraries

In [2]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd
import MySQLdb
import sqlalchemy as db
import pymongo

from sqlalchemy import create_engine

### MongoDB Credentials and Function

In [3]:
mongo_uid = "npa5gv"
mongo_pwd = "Greyscott23"
mongo_cluster = "cluster0.ylwahu7"

conn_str = {"local" : f"mongodb://localhost:27017/",
    "atlas" : f"mongodb+srv://{mongo_uid}:{mongo_pwd}@{mongo_cluster}.mongodb.net"
}


db_name = "med"

In [4]:
def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe

### Connecting to MongoDB

In [6]:
client = pymongo.MongoClient(conn_str["atlas"], tlsCAFile=certifi.where())
db = client[db_name]

# Gets the path of the Current Working Directory for this Notebook, and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd())

json_files = {"patients" : 'patients.json',
              "appointments" : 'appointments.json',
              "careplans" : 'careplans.json',
              "conditions" : 'conditions.json',
              "immunizations" : 'immunizations.json',
              "medications" : 'medications.json',
              "procedures" : 'procedures.json',
             }

for file in json_files:
    db.drop_collection(file)
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)

        
client.close()        

### Extracting Data

In [9]:
query = {} 
collection = "appointments"

df_appointments = get_mongo_dataframe(conn_str['atlas'], db_name, collection, query)
df_appointments.head(2)

,ID,DATE,PATIENT,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,5114a5b4-64b8-47b2-82a6-0ce24aae0943,2008-03-11,71949668-1c2e-43ae-ab0a-64654608defb,185349003,Outpatient Encounter,,
1,4d451e22-a354-40c9-8b33-b6126158666d,2009-01-11,71949668-1c2e-43ae-ab0a-64654608defb,185345009,Encounter for symptom,10509002,Acute bronchitis (disorder)


In [10]:
query = {} 
collection = "patients"

df_patients = get_mongo_dataframe(conn_str['atlas'], db_name, collection, query)
df_patients.head(2)

,patient,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,birthplace,address
0,4ee2c837-e60f-4c54-9fdf-8686bc70760b,4/8/1929,11/11/2029,999-78-5976,,,,Rosamaria,Pfannerstill,,,,black,dominican,F,Pittsfield MA US,18797 Karson Burgs Suite 444 Palmer Town MA 01...
1,efaf74f9-3de3-45dd-a5d5-26d08e8a3190,12/15/2016,2/19/2020,999-59-9186,,,,Loan,Bashirian,,,,white,american,F,Medford MA US,301 Eula Radial Suite 298 Brockton MA 02305 US


In [27]:
query = {} 
collection = "careplans"

df_careplans = get_mongo_dataframe(conn_str['atlas'], db_name, collection, query)
df_careplans.head(2)

,ID,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,e031962d-d13d-4ede-a449-040417d5e4fb,2009-01-11,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,53950000,Respiratory therapy,10509002,Acute bronchitis (disorder)
1,e031962d-d13d-4ede-a449-040417d5e4fb,2009-01-11,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,304510005,Recommendation to avoid exercise,10509002,Acute bronchitis (disorder)


In [12]:
query = {} 
collection = "conditions"

df_conditions = get_mongo_dataframe(conn_str['atlas'], db_name, collection, query)
df_conditions.head(2)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2009-01-08,2009-01-21,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,10509002,Acute bronchitis (disorder)
1,2010-10-16,2010-10-23,71949668-1c2e-43ae-ab0a-64654608defb,bed7ecff-b41c-422b-beac-ea00c8b02837,38822007,Cystitis


In [13]:
query = {} 
collection = "immunizations"

df_immunizations = get_mongo_dataframe(conn_str['atlas'], db_name, collection, query)
df_immunizations.head(2)

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2008-03-11,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,140,Influenza seasonal injectable preservative ...
1,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,bdb926b8-5b6d-4366-bbe9-76fe3f3fbc4f,140,Influenza seasonal injectable preservative ...


In [21]:
query = {} 
collection = "procedures"

df_procedure = get_mongo_dataframe(conn_str['atlas'], db_name, collection, query)
df_procedure.head(2)

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,2013-02-09,71949668-1c2e-43ae-ab0a-64654608defb,6f2e3935-b203-493e-a9c0-f23e847b9798,23426006,Measurement of respiratory function (procedure),10509002,Acute bronchitis (disorder)
1,2013-10-19,71949668-1c2e-43ae-ab0a-64654608defb,da4fd626-e74e-4930-91be-7fb3da7ea098,252160004,Standard pregnancy test,72892002,Normal pregnancy


In [15]:
query = {} 
collection = "medications"

df_medications = get_mongo_dataframe(conn_str['atlas'], db_name, collection, query)
df_medications.head(2)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,1988-09-05,,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,834060,Penicillin V Potassium 250 MG,43878008,Streptococcal sore throat (disorder)
1,2007-06-04,2008-06-04,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,1367439,NuvaRing 0.12/0.015 MG per 24HR 21 Day Vaginal...,,


### Transforming the Data

In [16]:
drop_cols = ['drivers', 'passport', 'suffix', 'marital']
df_patients.drop(drop_cols, axis=1, inplace=True)

In [17]:
df_patients.insert(0, "patient_key", range(1, df_patients.shape[0]+1))
df_patients.rename(columns={"patient":"patient_id", 
                            "birthdate": "birth_date",
                           "deathdate": "death_date"}, inplace=True)
df_patients.head(2)

,patient_key,patient_id,birth_date,death_date,ssn,prefix,first,last,maiden,race,ethnicity,gender,birthplace,address
0,1,4ee2c837-e60f-4c54-9fdf-8686bc70760b,4/8/1929,11/11/2029,999-78-5976,,Rosamaria,Pfannerstill,,black,dominican,F,Pittsfield MA US,18797 Karson Burgs Suite 444 Palmer Town MA 01...
1,2,efaf74f9-3de3-45dd-a5d5-26d08e8a3190,12/15/2016,2/19/2020,999-59-9186,,Loan,Bashirian,,white,american,F,Medford MA US,301 Eula Radial Suite 298 Brockton MA 02305 US


In [22]:
drop_cols = ['REASONCODE']
df_procedure.drop(drop_cols, axis=1, inplace=True)

In [23]:
df_procedure.insert(0, "procedure_key", range(1, df_procedure.shape[0]+1))
df_procedure.rename(columns={"DATE":"procedure_date",
                             "PATIENT":"patient_id", 
                             "CODE":"procedure_id",
                             "REASONDESCRIPTION":"procedure_reason",
                             "DESCRIPTION":"procedure_name",
                            "ENCOUNTER":"appointment_id",}, inplace=True)
df_procedure.head(2)

,procedure_key,procedure_date,patient_id,appointment_id,procedure_id,procedure_name,procedure_reason
0,1,2013-02-09,71949668-1c2e-43ae-ab0a-64654608defb,6f2e3935-b203-493e-a9c0-f23e847b9798,23426006,Measurement of respiratory function (procedure),Acute bronchitis (disorder)
1,2,2013-10-19,71949668-1c2e-43ae-ab0a-64654608defb,da4fd626-e74e-4930-91be-7fb3da7ea098,252160004,Standard pregnancy test,Normal pregnancy


In [28]:
drop_cols = ['CODE', 'REASONCODE']
df_careplans.drop(drop_cols, axis=1, inplace=True)

In [29]:
df_careplans.insert(0, "careplan_key", range(1, df_careplans.shape[0]+1))
df_careplans.rename(columns={"ID":"careplan_id",
                            "START":"care_start_date",
                            "STOP":"care_stop_date",
                            "PATIENT":"patient_id",
                            "ENCOUNTER":"appointment_id",
                             "DESCRIPTION":"careplan_name",
                           "REASONDESCRIPTION":"care_reason" }, inplace=True)
df_careplans.head(2)

,careplan_key,careplan_id,care_start_date,care_stop_date,patient_id,appointment_id,careplan_name,care_reason
0,1,e031962d-d13d-4ede-a449-040417d5e4fb,2009-01-11,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,Respiratory therapy,Acute bronchitis (disorder)
1,2,e031962d-d13d-4ede-a449-040417d5e4fb,2009-01-11,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,Recommendation to avoid exercise,Acute bronchitis (disorder)


In [30]:
df_conditions.insert(0, "condition_key", range(1, df_conditions.shape[0]+1))
df_conditions.rename(columns={"START":"cond_start_date", 
                             "STOP":"cond_stop_date",
                             "PATIENT":"patient_id",
                            "ENCOUNTER":"appointment_id",
                            "CODE":"condition_id",
                            "DESCRIPTION": "condition_name"}, inplace=True)

df_conditions.head(2)

,condition_key,cond_start_date,cond_stop_date,patient_id,appointment_id,condition_id,condition_name
0,1,2009-01-08,2009-01-21,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,10509002,Acute bronchitis (disorder)
1,2,2010-10-16,2010-10-23,71949668-1c2e-43ae-ab0a-64654608defb,bed7ecff-b41c-422b-beac-ea00c8b02837,38822007,Cystitis


In [31]:
drop_cols = ['CODE', 'REASONCODE']
df_appointments.drop(drop_cols, axis=1, inplace=True)

In [32]:
df_appointments.insert(0, "appointment_key", range(1, df_appointments.shape[0]+1))
df_appointments.rename(columns={"ID":"appointment_id",
                             "DATE": "appointment_date",
                              "DESCRIPTION": "appointment_type",
                              "PATIENT":"patient_id",
                              "REASONDESCRIPTION":"appointment_reason"}, inplace=True)

df_appointments.head(2)

,appointment_key,appointment_id,appointment_date,patient_id,appointment_type,appointment_reason
0,1,5114a5b4-64b8-47b2-82a6-0ce24aae0943,2008-03-11,71949668-1c2e-43ae-ab0a-64654608defb,Outpatient Encounter,
1,2,4d451e22-a354-40c9-8b33-b6126158666d,2009-01-11,71949668-1c2e-43ae-ab0a-64654608defb,Encounter for symptom,Acute bronchitis (disorder)


In [33]:
df_immunizations.insert(0, "immunization_key", range(1, df_immunizations.shape[0]+1))
df_immunizations.rename(columns={"CODE":"immunization_id",
                                "DATE": "immunization_date",
                                "PATIENT":"patient_id",
                                "ENCOUNTER":"appointment_id",
                                "DESCRIPTION":"immunization_name"}, inplace=True)

df_immunizations.head(2)

,immunization_key,immunization_date,patient_id,appointment_id,immunization_id,immunization_name
0,1,2008-03-11,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,140,Influenza seasonal injectable preservative ...
1,2,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,bdb926b8-5b6d-4366-bbe9-76fe3f3fbc4f,140,Influenza seasonal injectable preservative ...


In [34]:
drop_cols = ['REASONCODE']
df_medications.drop(drop_cols, axis=1, inplace=True)

In [35]:
df_medications.insert(0, "medication_key", range(1, df_medications.shape[0]+1))
df_medications.rename(columns={"CODE":"medication_id",
                               "START":"med_start_date",
                               "STOP":"med_stop_date",
                               "PATIENT":"patient_id",
                               "ENCOUNTER":"appointment_id",
                              "DESCRIPTION":"prescription",
                              "REASONDESCRIPTION": 'med_reason'}, inplace=True)

df_medications.head(2)

,medication_key,med_start_date,med_stop_date,patient_id,appointment_id,medication_id,prescription,med_reason
0,1,1988-09-05,,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,834060,Penicillin V Potassium 250 MG,Streptococcal sore throat (disorder)
1,2,2007-06-04,2008-06-04,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,1367439,NuvaRing 0.12/0.015 MG per 24HR 21 Day Vaginal...,


### MySQL Credentials and Functions

In [36]:
mysql_uid = "root"
mysql_pwd = "Greyscott23!"
mysql_hostname = "127.0.0.1"

db_name = "med_dw"
dst_dbname = "med"

In [37]:
def get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, db_name, sql_query):
    conn_str = f"mysql://{mysql_uid}:{mysql_pwd}@{mysql_hostname}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [38]:
def set_dataframe(mysql_uid, mysql_pwd, mysql_hostname, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql://{mysql_uid}:{mysql_pwd}@{mysql_hostname}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

## Loading into MySQL

In [40]:
db_operation = "insert"

tables = [('dim_procedures', df_procedure, 'procedure_key'),
          ('dim_careplans', df_careplans, 'careplan_key'),
          ('dim_conditions', df_conditions, 'condition_key'),
          ('dim_appointments', df_appointments, 'appointment_key'),
          ('dim_immunizations', df_immunizations, 'immunization_key'),
          ('dim_medication', df_medications, 'medication_key'),
          ('dim_patients', df_patients, 'patient_key')]

In [41]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, dataframe, table_name, primary_key, db_operation)

### Validating the New Tables

In [43]:
sql_procedures = "SELECT * FROM med.dim_procedures;"
df_dim_procedures = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_procedures)
df_dim_procedures.head(2)

,procedure_key,procedure_date,patient_id,appointment_id,procedure_id,procedure_name,procedure_reason
0,1,2013-02-09,71949668-1c2e-43ae-ab0a-64654608defb,6f2e3935-b203-493e-a9c0-f23e847b9798,23426006,Measurement of respiratory function (procedure),Acute bronchitis (disorder)
1,2,2013-10-19,71949668-1c2e-43ae-ab0a-64654608defb,da4fd626-e74e-4930-91be-7fb3da7ea098,252160004,Standard pregnancy test,Normal pregnancy


In [44]:
sql_careplans = "SELECT * FROM med.dim_careplans;"
df_dim_careplans = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_careplans)
df_dim_careplans.head(2)

,careplan_key,careplan_id,care_start_date,care_stop_date,patient_id,appointment_id,careplan_name,care_reason
0,1,e031962d-d13d-4ede-a449-040417d5e4fb,2009-01-11,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,Respiratory therapy,Acute bronchitis (disorder)
1,2,e031962d-d13d-4ede-a449-040417d5e4fb,2009-01-11,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,Recommendation to avoid exercise,Acute bronchitis (disorder)


In [45]:
sql_conditions = "SELECT * FROM med.dim_conditions;"
df_dim_conditions = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_conditions)
df_dim_conditions.head(2)

,condition_key,cond_start_date,cond_stop_date,patient_id,appointment_id,condition_id,condition_name
0,1,2009-01-08,2009-01-21,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,10509002,Acute bronchitis (disorder)
1,2,2010-10-16,2010-10-23,71949668-1c2e-43ae-ab0a-64654608defb,bed7ecff-b41c-422b-beac-ea00c8b02837,38822007,Cystitis


In [46]:
sql_appointments = "SELECT * FROM med.dim_appointments;"
df_dim_appointments = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_appointments)
df_dim_appointments.head(2)

,appointment_key,appointment_id,appointment_date,patient_id,appointment_type,appointment_reason
0,1,5114a5b4-64b8-47b2-82a6-0ce24aae0943,2008-03-11,71949668-1c2e-43ae-ab0a-64654608defb,Outpatient Encounter,
1,2,4d451e22-a354-40c9-8b33-b6126158666d,2009-01-11,71949668-1c2e-43ae-ab0a-64654608defb,Encounter for symptom,Acute bronchitis (disorder)


In [47]:
sql_immunizations = "SELECT * FROM med.dim_immunizations;"
df_dim_immunizations = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_immunizations)
df_dim_immunizations.head(2)

,immunization_key,immunization_date,patient_id,appointment_id,immunization_id,immunization_name
0,1,2008-03-11,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,140,Influenza seasonal injectable preservative ...
1,2,2009-04-07,71949668-1c2e-43ae-ab0a-64654608defb,bdb926b8-5b6d-4366-bbe9-76fe3f3fbc4f,140,Influenza seasonal injectable preservative ...


In [48]:
sql_medication = "SELECT * FROM med.dim_medication;"
df_dim_medication = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_medication)
df_dim_medication.head(2)

,medication_key,med_start_date,med_stop_date,patient_id,appointment_id,medication_id,prescription,med_reason
0,1,1988-09-05,,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,834060,Penicillin V Potassium 250 MG,Streptococcal sore throat (disorder)
1,2,2007-06-04,2008-06-04,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,1367439,NuvaRing 0.12/0.015 MG per 24HR 21 Day Vaginal...,


In [49]:
sql_patients = "SELECT * FROM med.dim_patients;"
df_dim_patients = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_patients)
df_dim_patients.head(2)

,patient_key,patient_id,birth_date,death_date,ssn,prefix,first,last,maiden,race,ethnicity,gender,birthplace,address
0,1,4ee2c837-e60f-4c54-9fdf-8686bc70760b,4/8/1929,11/11/2029,999-78-5976,,Rosamaria,Pfannerstill,,black,dominican,F,Pittsfield MA US,18797 Karson Burgs Suite 444 Palmer Town MA 01...
1,2,efaf74f9-3de3-45dd-a5d5-26d08e8a3190,12/15/2016,2/19/2020,999-59-9186,,Loan,Bashirian,,white,american,F,Medford MA US,301 Eula Radial Suite 298 Brockton MA 02305 US


## SQL Queries

### Query 1:

This query finds the procedures with the most occurences and returns the name of the procedure and the number of times it was performed. This information could be used for supply purchasing at a clinic: knowing the procedures that occur the most allows for preparation with the proper number of supplies.

In [75]:
sql_pop_procedures = "SELECT procedure_name, COUNT(*) AS procedure_count FROM dim_procedures GROUP BY procedure_name ORDER BY procedure_count DESC;"
query_1 = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_pop_procedures)
query_1.head(10)

,procedure_name,procedure_count
0,Documentation of current medications,3807
1,Intramuscular injection,1289
2,Subcutaneous immunotherapy,1281
3,Colonoscopy,449
4,Standard pregnancy test,287
5,Spirometry (procedure),255
6,Measurement of respiratory function (procedure),250
7,Cognitive and behavioral therapy,190
8,Combined chemotherapy and radiation therapy (p...,187
9,Suture open wound,167


### Query 2:

This query shows the overall diversity within the set of patient data, including the number of the most represented ethnicities. Because this is a synthetic set of data, it could be potentially used for medical students, so high diversity rates and inclusion is important for the learning process of future medical professionals (even in fake data) .

In [61]:
sql_diversity = "SELECT ethnicity, COUNT(*) AS population_count FROM dim_patients GROUP BY ethnicity ORDER BY population_count DESC;"
query_2 = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_diversity)
query_2.head(15)

,ethnicity,population_count
0,irish,306
1,italian,164
2,english,146
3,puerto_rican,116
4,german,80
5,french,79
6,polish,65
7,chinese,56
8,african,55
9,portuguese,55


### Query 3:

This query displays the average number of appointments attented by female patients and male patients. On average, the female patients from the data attend about 7 more appointments than the males. This could be due to the several pre-natal appointments that each pregnant woman in the data set schedules.

In [72]:
sql_gender = "WITH AppointmentsPerPatient AS (SELECT p.gender, a.patient_id, COUNT(a.appointment_id) AS appointment_count FROM dim_appointments a JOIN dim_patients p ON a.patient_id = p.patient_id GROUP BY p.gender, a.patient_id) SELECT 'Female' AS gender, AVG(appointment_count) AS avg_appointments FROM AppointmentsPerPatient WHERE gender = 'F' UNION SELECT 'Male' AS gender, AVG(appointment_count) AS avg_appointments FROM AppointmentsPerPatient WHERE gender = 'M';"
query_3 = get_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_gen)
query_3.head(15)

,gender,avg_appointments
0,Female,17.4514
1,Male,10.7409


# Conclusion

This notebook was successful in creating an Extract-Transform-Load Pipeline from local source data to MongoDB to MySQL Workbench. The queries executed show several ways that this data can exctract meaningful information about each patient through code.

## Sources

Original Data (Kaggle): https://www.kaggle.com/datasets/imtkaggleteam/synthetic-medical-dataset?resource=download&select=patients.csv